# Rabin-Miller
A friend of mine asked me to implement RSA. Let’s implement RSA from scratch, then. And since RSA encryption requires more than one algorithm, I will spread it into several days.

In this article I will dig deep but still not deep enough. I’m sorry if the explanation is not crystal clear or sufficient. I need to assume you are comfortable with basics of algebra. If you are not, just say loudly, “bad, bad math”, and read only the first and the last paragraphs.

> disclaimer: do not consider my code to be secure; do not consider any cryptography coming from non-experts to be secure; you should never implement any kind of cryptography on your own nor should you interfere with your security in any way; this series is just for fun and as such should be taken

First of all, I will need large prime numbers. To find some, I will implement [Rabin-Miller](https://en.wikipedia.org/wiki/Miller%E2%80%93Rabin_primality_test) which is what is called a strong pseudoprime test.

That’s a probabilistic test that always identifies a prime number, but sometimes incorrectly denotes a composite as prime. Fortunately, the chance to make a mistake can be decreased by repeating the test.

How does is work? Remember the famous [Fermat’s little theorem](https://en.wikipedia.org/wiki/Fermat%27s_little_theorem).

$$a^{p-1}\equiv1(mod p)$$

For a prime p the congruence forms a [finite field](https://en.wikipedia.org/wiki/Finite_field) and holds for any $a < p$. This has a consequence for a square roots of unity.

$$x^2\equiv1(mod p)$$
$$(x+1)(x-1)\equiv0(modp)$$
$$x\equiv\pm1(modp)$$

If $x^2$ is congruent to $1$, then $x+1$ or $x-1$ has to be divisible by $p$ which implies that $x$ must either be $1$ or $-1$. Hence for prime $p$ there exists no non-trivial (other than $1$ or $-1$) square root of unity.

Rabin-Miller searches for such roots. It starts by $a^{p-1}$ and repetitively takes the square roots. If any non-trivial root is found, $p$ is composite.

The recipe as described would be difficult to implement. But it can be implemented in a probabilistic way. First, we decompose $p$.

$$p=q2^r+1,1 is odd$$

Next we choose a random $a$ and check the following conditions.

For any prime $p$ either the first condition holds or there exists $s$ to comply the second condition. If the conditions can’t be satisfied, $p$ is composite.

But if $p$ is in fact a composite, there is $frac{1}{4}$ chance it will pass the test. Therefore we select another a and repeat the test. After testing $k$ independent values for $a$, the chance for a mistake gets down to $4^{-k}$.

Notice the idea behind Rabin-Miller. The test initially assumes $p$ to be a prime. Then it searches for evidence it is not. If the evidence is not found, $p$ is a prime with high probability, also called pseudoprime.

In [1]:
from random import randrange

## algorithm

In [2]:
def rabin_miller(prime, tests):
    if prime < 5:
        return prime in [2, 3]
    
    # set: prime = q * 2**r + 1
    q, r = prime - 1, 0
    while not q & 1:
        q >>= 1
        r += 1

    # test repeatedly
    for _ in range(tests):
        a = randrange(2, prime - 1)

        # pass if: a**q == 1
        x = pow(a, q, prime)
        if x in [1, prime - 1]:
            continue

        # pass if: a**(q * 2**s) == -1, s < r
        for _ in range(r - 1):
            x = pow(x, 2, prime)
            if x == prime - 1:
                break
        else:
            return False

    return True

In [3]:
def prime(bits, tests):
    while True:
        # random number in [2**bits .. 2**(bits+1)-1]
        prime = (1 << bits) | randrange(1 << bits) | 1

        # primality test
        if rabin_miller(prime, tests):
            return prime

## primes

In [4]:
prime(8, 32)

463

In [5]:
prime(256, 32)

176630768832900860353857644826040400763045229990510748739330296899179187403687

In [6]:
prime(1024, 32)

352587266501150105865545614578883727875485787605889586448064741820464135938585362828807924347176708215657148854434053193502471374165734214662762815735051810403486490614294855012372940069714522176651181222230895706953570804182258111674173876858187443030278787920827625293880002516052344311792613388818051839589